# Simulating Chemical Reaction Networks with ppsim

``ppsim`` is able to simulate any Chemical Reaction Network (CRN) whose reactions are all bimolecular (two reactant, two product) or unimolecular (one reactant, one product). This notebook shows some examples of that feature, and how it compares to the standard Gillespie stochastic simulation algorithm.

In [1]:
import ppsim as pp
import numpy as np
from matplotlib import pyplot as plt
import pickle

In [2]:
%matplotlib widget

# Approximate majority CRN

This state CRN reaches a consensus between the two initial states ``A`` and ``B``. Opposite opinions become undecided ``U`` states, which are then converted by the ``A`` and ``B`` states.

In [35]:
a,b,u = pp.species('A B U')
approx_majority = [
    a+b >> 2*u,
    a+u >> 2*a,
    b+u >> 2*b,
]
n = 10 ** 2
p = 0.51
init_config = {a: p*n, b: (1-p)*n}
sim = pp.Simulation(init_config, approx_majority)
sim.run(30, 0.1)
sim.history.plot(figsize = (6,4))
plt.title('approximate majority protocol')
plt.show()

 Time: 30.000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

We will implement the same protocol using Gillespie simulation with the package [GillesPy2](https://github.com/StochSS/GillesPy2).

In [4]:
import gillespy2
model = pp.gillespy2_format(init_config, approx_majority, n)
model.timespan(np.linspace(0,30,300))
model.run().plot(figsize = (6, 4), title='Approximate Majority')
end_time = 5
plt.vlines(end_time, 0, n, color='r',linestyles = 'dashed')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

To test more thoroughly that these two simulations have the exact same distribution, we will sample the probability distribution of each state at time 5.

In [5]:
# trials = 10 ** 6
# ran 1 million trials for recorded data, but now trials is set lower so the notebook runs faster
trials = 10 ** 4
sim = pp.Simulation(init_config, approx_majority)
results_ppsim_cts = sim.sample_future_configuration(end_time, num_samples = trials)
# Run additional trials with discrete time for comparison
sim.continuous_time = False
results_ppsim_discrete = sim.sample_future_configuration(end_time, num_samples = trials)

model.timespan(np.linspace(0,end_time,2))
results_gillespy2 = model.run(number_of_trajectories = trials)

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

In [6]:
fig, axes = plt.subplots(1, 2)
for state, ax in zip(['A', 'B'], axes):
    ax.hist([results_ppsim_cts[state], [result[state][1] for result in results_gillespy2]], bins = np.linspace(0, n, 20), 
                              alpha = 1, label=['ppsim', 'gillespy2'], density=True, edgecolor = 'k', linewidth = 0.0)
    ax.legend()
    ax.set_xlabel(f'Count of {state}')
axes[0].set_ylabel('empirical probability')
fig.suptitle('Sampled state distributions')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0.98, 'Sampled state distributions')

Now we will take a look at larger population size, to see how the time for ``ppsim`` to run compares to ``gillespy2`` and also another Gillespie SSA simulator from the package [StochKit2](https://academic.oup.com/bioinformatics/article/27/17/2457/224105).

In [33]:
from time import perf_counter
from IPython.display import clear_output
import math
import os

def time_trials(simulator, ns, num_trials = 10000):
    # get data on time it takes ppsim to run rule from init_config until end_time
    times = []
    for n in ns:
        print(f'n = {n:.2E}')
        sim = simulator(n)
        start_time = perf_counter()
        sim.run(num_trials)
        trial_time = (perf_counter() - start_time) / num_trials
        times.append(trial_time)
        # set num_trials to be at most the number that this one could have finished in 10 seconds
        if trial_time > 1000:
            break
        num_trials = min(num_trials, math.ceil(10 / trial_time))
        clear_output(wait=True)
    return times

# redefining this function will change the rule that the trials use
def sim_params(n):
    # returns init_config, rxns, vol
    return {a: n // 2, b: n // 2}, approx_majority, n

class gillespy2_trials:
    def __init__(self, n):
        self.model = pp.gillespy2_format(*sim_params(n))
        self.model.timespan([0,end_time])
    def run(self, num_trials):
        self.model.run(number_of_trajectories=num_trials)

class ppsim_trials:
    def __init__(self, n):
        self.sim = pp.Simulation(*sim_params(n)[0:2])
    def run(self, num_trials):
        self.sim.sample_future_configuration(end_time, num_trials)
        
# In order to run stochkit_trials, need a stochkit package in the environment variable STOCHKIT_HOME
class stochkit_trials:
    def __init__(self, n):
        # save to build folder, which is in .gitignore
        self.filename = 'build/stochkit_trial.xml'
        init_config, rxns, vol = sim_params(n)
        pp.write_stochkit_file(self.filename, *sim_params(n))
        self.cmd = os.path.join(os.environ['STOCHKIT_HOME'], 'ssa')
    def run(self, num_trials):
        self.cmd += ' -m ' + self.filename + ' -r ' + str(num_trials) + ' -t ' + str(end_time) + ' -f'
        os.system(self.cmd)

In [42]:
# # Code that was used to generate the data, which was saved with pickle
# ns = [int(n) for n in np.geomspace(10 ** 1, 10 ** 12, 23)]
# end_time = 10
# num_trials = 1000
# methods = [ppsim_trials, gillespy2_trials, stochkit_trials]
# method_times2 = [time_trials(method, ns) for method in methods]
# pickle.dump(method_times, open( "crn_data/am_runtimes.p", "wb" ) )

In [47]:
# load the pickled data to plot
method_times = pickle.load( open( "crn_data/am_runtimes.p", "rb" ))
fig = plt.figure()
for times in method_times:
    plt.plot(ns[2:len(times)], times[2:])
plt.xscale('log')
plt.yscale('log')
plt.xlabel('population size')
plt.ylabel('running time (s)')
plt.xticks(ns[2::2])
plt.title('Average running time to run approximate majority to time 10')
plt.legend(['ppsim', 'GillesPy2', 'StochKit2'])
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [48]:
plt.savefig('crn_data/am3.svg')

The runtime plots are reflecting that Gillespie has $O(n)$ runtime (roughly slope 1 on log-log plot), wheras ``ppsim`` has $O(\sqrt{n})$ runtime (roughly slope 1/2 on log-log plot).

Using ``ppsim``, we are able to simulate populations into the trillions. This approximate majority protocol has been implemented experimentally by DNA strand displacement in [this paper](https://pubmed.ncbi.nlm.nih.gov/24077029/). We can set the approximate majority rate constants to be effective rate constants from the paper, and using the concentration 80 nM from the paper in a volume of 1 microliter, we have a system of nearly $10^{11}$ molecules, which can be ``ppsim`` can handle.

In [18]:
# CRN for approximate majority
k1,k2,k3 = 9028, 2945, 1815
total_concentration = 80 * 1e-9 # 1x volume was 80 nM
vol = 1e-6 # 1 uL
n = pp.concentration_to_count(total_concentration, vol)
approx_majority_rates = [
    (a+b >> 2*u).k(k1, units=pp.RateConstantUnits.mass_action),
    (a+u >> 2*a).k(k2, units=pp.RateConstantUnits.mass_action),
    (b+u >> 2*b).k(k3, units=pp.RateConstantUnits.mass_action),
]
# set the initial concentrations near where the the mass-action CRN would reach an unstable equilibrium
p = 0.45
init_config = {a: int(p*n), b: int((1-p)*n)}
# d = int(n / (k1 + k2 + k3))
# init_config = {a: k3 * d, b: k2 * d, u: k1* d}
sim = pp.Simulation(init_config, approx_majority_rates, volume=vol, time_units='seconds')
hp = pp.HistoryPlotter()
sim.add_snapshot(hp)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'approximate majority, experimental conditions')

In [25]:
%time sim.run()
hp.ax.set_title('approximate majority, experimental conditions')
hp.ax.set_ylabel('count')

Text(59.794445951779686, 0.5, 'count')

# Large State Speed Tests

We will now see how `ppsim`'s running time scales with the number of states. Our first example CRN will be a chain of unimolecular decay, where each of `m` states decays into the next.

In [75]:
def get_unimolecular_chain(m):
    # create reactions with m species s0, ..., s(m-1)
    states = pp.species(' '.join([str(i) for i in range(m)]))
    rxns = [states[i] >> states[i+1] for i in range(m-1)]
    return states, rxns
states, rxns = get_unimolecular_chain(10)
n = 10 ** 4
sim = pp.Simulation({states[0]: n}, rxns)
sim.run(history_interval=0.1)
sim.history.plot()
plt.ylabel('count')
plt.yscale('symlog')
plt.ylim(0, 2*n)
plt.title('unimolecular decay')

 Time: 24.500


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'unimolecular decay')

The plot with time on a linear scale and counts on a log scale shows each state successively decaying exponentially to 0.

We will compare ppsim with StochKit2, at a fixed population size $n = 10^4$ where they had comparable times with 3-state approximate majority. Unlike in the above plot, the initial configuration will be a uniform mix of all `m` states. This is to ensure that we can test for a fixed amount of time and have all reactions be active simulatenously.

In [65]:
n = 10 ** 4
end_time = 10

def sim_params(m):
    # returns init_config, rxns, vol
    states, rxns = get_unimolecular_chain(m)
    return {state:n // m for state in states}, rxns, n

In [66]:
# Code that was used to generate the data, which was saved with pickle
ms = [int(m) for m in np.linspace(2, 1000, 20)]
methods = [ppsim_trials, stochkit_trials]
method_times = [time_trials(method, ms, num_trials=100) for method in methods]
pickle.dump(method_times, open( "crn_data/unimolecular_runtimes.p", "wb" ) )

n = 1.00E+03


In [70]:
# load the pickled data to plot
method_times = pickle.load( open( "crn_data/unimolecular_runtimes.p", "rb" ))
fig = plt.figure()
for times in method_times:
    plt.plot(ms[:len(times)], times)
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel('number of states')
plt.ylabel('running time (s)')
plt.title('Average running time for unimolecular chain, n = 10^4')
plt.legend(['ppsim', 'StochKit2'])
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [76]:
plt.savefig('crn_data/unimolecular1.svg')

Next we will look at a chain where each reaction is bimolecular.

In [89]:
def get_bimolecular_chain(m):
    # create reactions with m species s0, ..., s(m-1)
    states = pp.species(' '.join([str(i) for i in range(m)]))
    rxns = [2*states[i] >> states[i] + states[i+1] for i in range(m-1)]
    return states, rxns
states, rxns = get_bimolecular_chain(10)
sim = pp.Simulation({states[0]: n}, rxns)
# Setting history_interval to be a function lets the recorded timesteps be evenly spaced on a log plot
sim.run(history_interval=lambda t: 10 ** len(str(int(t))) / 100)
sim.history.plot()
plt.xscale('symlog')
plt.xlim(0, sim.times[-1])
plt.yscale('symlog')
plt.ylabel('count')
plt.ylim(0, 2*n)
plt.title('bimolecular decay')

 Time: 115722.100


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'bimolecular decay')

Now a plot with time and counts on a log scale shows that each state successive decays polynomially to 0. Because `ppsim` dynamically switches to Gillespie's algorithm when the number of null interactions gets very high, it can quickly go out to large times if the system is waiting on a small number of reactions to complete.

In [103]:
n = 10 ** 4
end_time = 100

def sim_params(m):
    # returns init_config, rxns, vol
    states, rxns = get_bimolecular_chain(m)
    return {state:n // m for state in states}, rxns, n

In [104]:
# Code that was used to generate the data, which was saved with pickle
ms = [int(m) for m in np.linspace(2, 1000, 20)]
methods = [ppsim_trials, stochkit_trials]
method_times = [time_trials(method, ms, num_trials=100) for method in methods]
pickle.dump(method_times, open( "crn_data/bimolecular_runtimes.p", "wb" ) )

n = 1.00E+03


In [118]:
# load the pickled data to plot
method_times = pickle.load( open( "crn_data/bimolecular_runtimes.p", "rb" ))
ms = [int(m) for m in np.linspace(2, 1000, 20)]
fig = plt.figure()
for times in method_times:
    plt.plot(ms[:len(times)], times)
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel('number of states')
plt.ylabel('running time (s)')
plt.title('Average running time for bimolecular chain, n = 10^4')
plt.legend(['ppsim', 'StochKit2'])
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [123]:
def get_discrete_averaging(m):
    # create reactions with m species s0, ..., s(m-1)
    states = pp.species(' '.join([str(i) for i in range(m)]))
    rxns = []
    for i in range(m):
        for j in range(i):
            avg = (i + j) / 2
            rxns.append(states[i] + states[j] >> states[math.floor(avg)] + states[math.ceil(avg)])
    return states, rxns

In [124]:
n = 10 ** 4
end_time = 10

def sim_params(m):
    # returns init_config, rxns, vol
    states, rxns = get_discrete_averaging(m)
    return {states[0]: n // 2, states[m-1]: n // 2}, rxns, n

In [125]:
# Code that was used to generate the data, which was saved with pickle
ms = [int(m) for m in np.linspace(3, 1000, 20)]

methods = [ppsim_trials, stochkit_trials]
method_times = [time_trials(method, ms, num_trials=100) for method in methods]
pickle.dump(method_times, open( "crn_data/discrete_averaging_runtimes.p", "wb" ) )

n = 5.27E+02


In [110]:
# load the pickled data to plot
method_times = pickle.load( open( "crn_data/discrete_averaging_runtimes.p", "rb" ))
fig = plt.figure()
for times in method_times:
    plt.plot(ms[:len(times)], times)
# plt.xscale('log')
# plt.yscale('log')
plt.xlabel('number of states')
plt.ylabel('running time (s)')
plt.title('Average running time for discrete averaging, n = 10^4')
plt.legend(['ppsim', 'StochKit2'])
plt.grid()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# RPS Oscillator

In [111]:
# CRN for rps oscillator
k1,k2,k3 = 1, 1, 1
a,b,c = pp.species('A B C')
rps = [
    (b+a >> 2*b).k(k1),
    (c+b >> 2*c).k(k2),
    (a+c >> 2*a).k(k3),
]

In [112]:
n = 10 ** 10
dist = [0.3, 0.3, 0.4]
init_config = {a: n * dist[0], b: n * dist[1], c: n * dist[2]}
sim = pp.Simulation(init_config, rps)
sim.run(100, 0.1)
sim.history.plot(figsize = (6,4))
plt.title('rps oscillator')

 Time: 100.000


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'rps oscillator')

In [113]:
sim.run()
plt.figure()
sim.history['A'].plot(figsize = (6,4))
plt.title('rps oscillator')

KeyboardInterrupt: 

In [ ]:
n = 10 ** 10
init_config = {a: n // 3, b: n // 3, c: n // 3}
sim = pp.Simulation(init_config, rps)

In [ ]:
hp=pp.HistoryPlotter(update_time = 1)
sim.add_snapshot(hp)

In [ ]:
sim.run(history_interval = 0.1)

In [ ]:
class RPS(gillespy2.Model):
     def __init__(self, n):
            #initialize Model
            gillespy2.Model.__init__(self, name="RPS_Oscillator")
            
            self.volume = n
            
            rate1 = gillespy2.Parameter(name='rate1', expression= k1)
            rate2 = gillespy2.Parameter(name='rate2', expression= k2)
            rate3 = gillespy2.Parameter(name='rate3', expression = k3)
            
            # Add parameters to the model
            self.add_parameter([rate1,rate2,rate3])
            
            A = gillespy2.Species(name='A', initial_value= n * dist[0])
            B = gillespy2.Species(name='B', initial_value= n * dist[1])
            C = gillespy2.Species(name='C', initial_value= n * dist[2])
            
            # Add species to the model
            self.add_species([A, B, C])
            
            r1 = gillespy2.Reaction(name="r1",reactants={A:1,B:1}, products={B:2},
                   rate=rate1)
            
            r2 = gillespy2.Reaction(name="r2",reactants={B:1, C:1}, products={C:2},
                    rate=rate2)
            
            r3 = gillespy2.Reaction(name="r3",reactants={A:1, C:1}, products={A:2},
                    rate=rate3)
            
            # Add reactions to the model
            self.add_reaction([r1,r2,r3])
            
            # Set timespan of model
            self.timespan(np.linspace(0,1000,10000))

In [ ]:
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
model = RPS(n)
results=model.run()
results.plot(figsize = (6, 4))

def phase_plot(results):
    ax = plt.figure().add_subplot(projection='3d')
    verts = [[n,0,0],[0,n,0],[0,0,n]]
    simplex = Poly3DCollection(verts, edgecolors='k', facecolors='w', alpha=0.1)
    ax.add_collection3d(simplex)

    ax.plot(results['A'], results['B'], results['C'], lw=0.5)
    ax.set_xlabel('A')
    ax.set_ylabel('B')
    ax.set_zlabel('C')
    ax.set_xlim(0, n)
    ax.set_ylim(0, n)
    ax.set_zlim(0, n)
    ax.set_title("rps oscillator")
    ax.grid(False)
    
    ax.scatter(results['A'][0], results['B'][0], results['C'][0], c='g', edgecolor='k')
    ax.scatter(results['A'][-1], results['B'][-1], results['C'][-1], c='r', edgecolor='k')
    plt.show()
    return(ax)
    
ax = phase_plot(results)

In [ ]:
model = RPS(n)
results=model.run(solver=gillespy2.TauLeapingSolver)
results.plot(figsize = (6, 4))
ax = phase_plot(results)
ax.set_title("rps oscillator with tau-leaping")

In [ ]:
# For some reason gillespy2.ODESolver doesn't work
# model = RPS(3)
# model.timespan(np.linspace(0,200,2000))
results=model.run(solver=gillespy2.ODESolver)
results.plot(figsize = (6, 4))
ax = plt.figure().add_subplot(projection='3d')
ax.plot(results['A'], results['B'], results['C'])
ax.set_xlabel('A')
ax.set_ylabel('B')
ax.set_zlabel('C')
ax.set_title("rps oscillator")
plt.show()